In [1]:
import nltk
import numpy
import pandas
import sklearn

import sqlite3

import csshelper
import cssfeature
import csspipe
import csstransformer


## Load data
(run bash ./src/init.sh first!)

In [3]:
#data = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
#help(data.head)
#data.head(1)
db_path = './data.sqlite3'
sqlite_connection = sqlite3.connect(db_path)

cNeu_reader = csshelper.CNeuReader(sqlite_connection)
sNeu_reader = csshelper.SNeuReader(sqlite_connection)

print(cNeu_reader.get_results().head(3))
print(sNeu_reader.get_results().head(3))



                                            sentence  cNeu
0                        likes the sound of thunder.  True
1  is so sleepy it's not even funny that's she ca...  True
2  is sore and wants the knot of muscles at the b...  True
                                            sentence  sNeu
0                        likes the sound of thunder.     3
1  is so sleepy it's not even funny that's she ca...     3
2  is sore and wants the knot of muscles at the b...     3


# Create DataFrame with string-length-featues

In [4]:

## create length of sentence feature
#sl_data = cssfeature.CharacterFeatures.string_length(cNeu_reader.get_results(), column=0, column_name='sentence_length')

#print(sl_data.head(2))


In [5]:
features = sl_data[['sentence_length']].astype(str)
labels = sl_data[['cNeu']].astype(bool)

print(features.head(2))
print(labels.head(2))

NameError: name 'sl_data' is not defined

In [8]:

split = sklearn.cross_validation.train_test_split(features, labels, train_size = 0.66, random_state= 6432119)

train_feat, test_feat, train_label, test_label = split

"""
print("train_feat\n%s\n" %train_feat.head(1))
print("train_label\n%s\n" %train_label.head(1))
print("test_feat\n%s\n" %test_feat.head(1))
print("test_label\n%s\n" %test_label.head(1))
"""


NameError: name 'features' is not defined

# Create pipefactory

In [6]:
pipefactory = csspipe.PipeFactory()

### Create classifiers for testing

In [7]:

multinomial_nb = pipefactory.classifiers.multinomial_nb
bernoulli_nb = pipefactory.classifiers.bernoulli_nb

svc = pipefactory.classifiers.svc
linear_svc = pipefactory.classifiers.linear_svc

random_forest = pipefactory.classifiers.random_forest_classifier
k_neighbors = pipefactory.classifiers.k_neighors_classifier


## Create extractors for testing

In [8]:

tfidf_vectorizer = pipefactory.extractors.tfidf_vectorizer


## Create selectors for testing

In [9]:
k_best = pipefactory.selectors.select_k_best

In [10]:
mytagger = csstransformer.PartOfSpeech(ignore_unknown=True)

In [11]:
#postagger.evaluate(treebank.tagged_sents())
input = cNeu_reader.get_results().sentence[:5,]
print()
print(mytagger.transform(input))
print()
#print(mytagger.transform(input))

TypeError: create() missing 1 required positional argument: 'tokenizer'

## Findings:
-  ngrams ranging from 2-3 return better results than 1-1
- full Part-Of-Speech of a sentence + tfidf does not behave well

###Why pos tagging failed:
We used tfidf to build vectors. tfidf's purpose however, is to *minimize the impact of frequently repeting components* (words). Thus the "part of speech"-equivalent of a sentence consisting of a description such as 'NN' will have less impact than a sentence build out of real words, as they are way less frequent in the whole test data.


###POS-Workarounds
- instead of converting a full sentence into POS-Tag one could try to filter all nouns or adjectives


In [16]:
data_reader_collection = [
        ('cExt', csshelper.CExtReader(sqlite_connection))
        , ('sExt', csshelper.SExtReader(sqlite_connection))
    
        , ('cNeu', csshelper.CNeuReader(sqlite_connection))
        , ('sNeu', csshelper.SNeuReader(sqlite_connection))
     
        , ('cAgr', csshelper.CAgrReader(sqlite_connection))
        , ('sAgr', csshelper.SAgrReader(sqlite_connection))
    
        , ('cCon', csshelper.CConReader(sqlite_connection))
        , ('sCon', csshelper.SConReader(sqlite_connection))
    
        , ('cOpn', csshelper.COpnReader(sqlite_connection))
        , ('sOpn', csshelper.SOpnReader(sqlite_connection))
    
]

aggregated_data_reader_collection = [
        ('cExt_aggregated', csshelper.CExtAggregatedReader(sqlite_connection))
    
        , ('cNeu_aggregated', csshelper.CNeuAggregatedReader(sqlite_connection))
     
        , ('cAgr_aggregated', csshelper.CAgrAggregatedReader(sqlite_connection))
    
        , ('cCon_aggregated', csshelper.CConAggregatedReader(sqlite_connection))
    
        , ('cOpn_aggregated', csshelper.COpnAggregatedReader(sqlite_connection))
]

In [17]:
pipeline_collection = [
    # 3 Support Vector Machines
    ('classifierLinearSVC', pipefactory.create_pipe([
        ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer()),
        ('clf', sklearn.svm.LinearSVC(random_state = 5152))]) # OneVsRestClassifier already implemented, docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification
    )
    
    , ('classifierSVC', pipefactory.create_pipe([
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('clf', sklearn.svm.SVC(cache_size=500, random_state = 5152, kernel = "rbf", decision_function_shape = "ovr"))]) # "one-against-one", docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification
    )
    # 2 Naive Bayes
    , ('classifierNB', pipefactory.create_pipe([
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('nb', sklearn.naive_bayes.MultinomialNB())])
    )

    , ('classifierBNB', pipefactory.create_pipe([    
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('bnb', sklearn.naive_bayes.BernoulliNB())])
    )
    
    # Nearest Neighboars
    , ('classifier_kNN', pipefactory.create_pipe([
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('kNN', sklearn.neighbors.KNeighborsClassifier(n_jobs=-1))]) # http://scikit-learn.org/stable/modules/neighbors.html#neighbors
    )
    # 2 Ensamble methods
    , ('classifierRF', pipefactory.create_pipe([    
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('rfc', sklearn.ensemble.RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state = 5152))])
    )
    , ('classifierADC', pipefactory.create_pipe([    
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('adc', sklearn.ensemble.AdaBoostClassifier(random_state = 5152))])
    )
]

In [24]:
pipe = pipefactory.create_pipe([
        ('smileys', csstransformer.SmileyTransformer()),
        #('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,1),stop_words=None)),
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,1),vocabulary=csstransformer.SmileyTransformer.smileys, stop_words=None)),
        ('rfc', sklearn.ensemble.RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state = 5152))
        #('nb', sklearn.naive_bayes.MultinomialNB())
    ])
data = csshelper.COpnReader(sqlite_connection).get_results()
#data = csshelper.COpnAggregatedReader(sqlite_connection).get_results()

feature = data.sentence
label = data.cOpn

#print(csstransformer.SmileyTransformer().transform(label))
#print(csstransformer.NounTransformer().transform(label))

smiley_feature = csstransformer.SmileyTransformer().transform(feature)

tfidf = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,1),vocabulary=['<3',':D'], stop_words=None).fit(smiley_feature, label)

x = tfidf.transform(smiley_feature)

#print(x)
#print(x.toarray())

#print(smiley_feature)

print(sklearn.cross_validation.cross_val_score(pipe, feature, label, cv=10).mean())

0.743168455617


In [27]:
## Pipelineing
# http://scikit-learn.org/stable/modules/pipeline.html
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

raw_pipeline = pipefactory.create_pipe([
        #tfidf_vectorizer
        ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(2,3), analyzer='word'))
        #, ('kNN', sklearn.neighbors.KNeighborsClassifier())
        , multinomial_nb
    ])

pos_pipeline = pipefactory.create_pipe([
        #tfidf_vectorizer
        #('part of speech', Test())
        #('part of speech', csstransformer.PartOfSpeech(ignore_unknown=True))
        ('part of speech', csstransformer.NounTransformer())
        #, ('count vectorizer', sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1)))
        , ('tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(2,3), analyzer='word'))
        #, ('k_best', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000))
        #, ('kNN', sklearn.neighbors.KNeighborsClassifier())
        , multinomial_nb
    ])


for feature_name, reader in aggregated_data_reader_collection:
    print(feature_name)
    
    data = reader.get_results()
    
    label = data.iloc[:,0]
    feature = data.iloc[:,1]
    
    for pipe_name, pipeline in pipeline_collection:
        print("\t\033[1m%s\033[0m" % pipe_name)
        acc = sklearn.cross_validation.cross_val_score(pipeline, label, feature, cv=5)
        print("\t%s" % acc.mean())
        print()
        pass
    
    #r1 = sklearn.cross_validation.cross_val_score(raw_pipeline, label, feature, cv=5)
    #r2 = sklearn.cross_validation.cross_val_score(pos_pipeline, label, feature, cv=5)

    #print(r1.mean())
    #print(r2.mean())


cExt_aggregated
	classifierLinearSVC
	0.616099239696

	classifierSVC
	0.616017607043

	classifierNB
	0.616017607043

	classifierBNB
	0.61993917567

	classifier_kNN
	0.536092837135

	classifierRF
	0.592017607043

	classifierADC
	0.544949179672

cNeu_aggregated
	classifierLinearSVC
	0.580419367747

	classifierSVC
	0.604017607043

	classifierNB
	0.604017607043

	classifierBNB
	0.604017607043

	classifier_kNN
	0.580331332533

	classifierRF
	0.536321728691

	classifierADC
	0.540246498599

cAgr_aggregated
	classifierLinearSVC
	0.596416166467

	classifierSVC
	0.536004801921

	classifierNB
	0.540004801921

	classifierBNB
	0.532083233293

	classifier_kNN
	0.419342136855

	classifierRF
	0.512324929972

	classifierADC
	0.459988795518

cCon_aggregated
	classifierLinearSVC
	0.632

	classifierSVC
	0.52

	classifierNB
	0.628

	classifierBNB
	0.548

	classifier_kNN
	0.528

	classifierRF
	0.48

	classifierADC
	0.584

cOpn_aggregated
	classifierLinearSVC
	0.691870348139

	classifierSVC
	0.704033613445



In [26]:
## Pipelineing
# http://scikit-learn.org/stable/modules/pipeline.html
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
for feature_name, reader in aggregated_data_reader_collection:
    print(feature_name)
    
    data = reader.get_results()
    
    label = data.iloc[:,0]
    feature = data.iloc[:,1]
    
    label = csstransformer.StemmedWordsTransformer().transform(label)
    
    for pipe_name, pipeline in pipeline_collection:
        print("\t\033[1m%s\033[0m" % pipe_name)
        acc = sklearn.cross_validation.cross_val_score(pipeline, label, feature, cv=5)
        print("\t%s" % acc.mean())
        print()
        pass
    
    #r1 = sklearn.cross_validation.cross_val_score(raw_pipeline, label, feature, cv=5)
    #r2 = sklearn.cross_validation.cross_val_score(pos_pipeline, label, feature, cv=5)

    #print(r1.mean())
    #print(r2.mean())


cExt_aggregated
	classifierLinearSVC
	0.616099239696

	classifierSVC
	0.616017607043

	classifierNB
	0.616017607043

	classifierBNB
	0.61993917567

	classifier_kNN
	0.536092837135

	classifierRF
	0.564174469788

	classifierADC
	0.524557022809

cNeu_aggregated
	classifierLinearSVC
	0.576337735094

	classifierSVC
	0.604017607043

	classifierNB
	0.604017607043

	classifierBNB
	0.604017607043

	classifier_kNN
	0.576331332533

	classifierRF
	0.608661064426

	classifierADC
	0.556089635854

cAgr_aggregated
	classifierLinearSVC
	0.592416166467

	classifierSVC
	0.536004801921

	classifierNB
	0.540004801921

	classifierBNB
	0.532083233293

	classifier_kNN
	0.415342136855

	classifierRF
	0.468227290916

	classifierADC
	0.463988795518

cCon_aggregated
	classifierLinearSVC
	0.624

	classifierSVC
	0.52

	classifierNB
	0.624

	classifierBNB
	0.548

	classifier_kNN
	0.532

	classifierRF
	0.512

	classifierADC
	0.564

cOpn_aggregated
	classifierLinearSVC
	0.695870348139

	classifierSVC
	0.704033613445


In [22]:
x = ('classifierNB', pipefactory.create_pipe([
        ('vectorizer_tfidf', sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,2))),
        ('nb', sklearn.naive_bayes.MultinomialNB())])
    )
y = ('classifierNB', pipefactory.create_pipe([
        ('hashingvectorizer', sklearn.feature_extraction.text.HashingVectorizer()),
        ('tfidf transformer', sklearn.feature_extraction.text.TfidfTransformer()),
        ('nb', sklearn.naive_bayes.MultinomialNB())])
    )

cExt_data = csshelper.CExtAggregatedReader(sqlite_connection).get_results()

label = cExt_data.sentence
feature = cExt_data.cExt

for desc, pipeline in [x,y]:
    acc = sklearn.cross_validation.cross_val_score(pipeline, label, feature, cv=5)
    print("\t%s" % acc.mean())

	0.616017607043


ValueError: Input X must be non-negative

In [27]:
test_data = data['sentence'].astype(str).values[:3]

print("test_data:\n%s" % test_data)

tfidf_trans = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
#help(tfidf_trans)
tfidf_trans.fit(test_data)
#tfidf_trans.transform(test_data)

print(tfidf_trans.get_feature_names()[:10])

type(tfidf_trans.transform(data))
print(tfidf_trans.transform(["funny", "and"]).toarray())

#tfidf_trans.get_feature_names()

test_data:
['likes the sound of thunder.'
 "is so sleepy it's not even funny that's she can't get to sleep."
 "is sore and wants the knot of muscles at the base of her neck to stop hurting. On the other hand, YAY I'M IN ILLINOIS! <3"]
['and', 'and wants', 'at', 'at the', 'base', 'base of', 'can', 'can get', 'even', 'even funny']
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]


In [28]:
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()

pipeline = sklearn.pipeline.Pipeline([('tfidf', tfidf_vectorizer),
                     ('k_best', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000)),
                     ('nb', sklearn.naive_bayes.MultinomialNB())
                    ])

#pipeline.fit(train_feat[["STATUS"]], train_label[["cNEU"]])
#data = train_feat.STATUS
#target = train_label.cNEU.apply(lambda x: x == 'y')

#foo = tfidf_vectorizer.fit_transform(data[:1], target[:1])
#print(foo)
#idf = foo.idf_

#print(len(idf))
#print(len(data))

#pipeline.fit(data.sentence, data.cNeu)
import sklearn
import sklearn.feature_extraction.text

data = csshelper.CNeuReader(sqlite_connection).get_results()

features = data.sentence
labels = data.cNeu

cv = sklearn.feature_extraction.text.CountVectorizer()

cv_fit = cv.fit(features, labels)
cv_transform_features = cv_fit.transform(features)


tfidftrans = sklearn.feature_extraction.text.TfidfTransformer()

tfidftrans_fit = tfidftrans.fit(cv_transform_features, labels)
tfidftrans_transform_features = tfidftrans_fit.transform(cv_transform_features)

#print(cv_transform_features)
#print(tfidftrans_transform_features)

skb = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000)
mnb = sklearn.naive_bayes.MultinomialNB()



mnb_fit = mnb.fit(tfidftrans_transform_features, labels.astype(bool))

print(hasattr(mnb_fit, "fit"))
print(hasattr(mnb_fit, "transform"))
print(hasattr(mnb_fit, "fit_transform"))

pipe = sklearn.pipeline.Pipeline([
    ('cv', sklearn.feature_extraction.text.CountVectorizer())
    , ('tfidf_trans', sklearn.feature_extraction.text.TfidfTransformer())
    , ('nb', sklearn.naive_bayes.MultinomialNB())
])

sklearn.cross_validation.cross_val_score(pipe, features.astype(str), labels.astype(bool))



#print(tfidf_vectorizer.fit_transform(features, labels))
#tfidf = tfidf_vectorizer.fit_transform(features, labels)
#print(tfidf)

#print(tfidf[[9]])


select_k = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.chi2, k=1000)
print('select_k has transform: %s' % hasattr(select_k, 'transform'))
print('select_k has fit: %s' % hasattr(select_k, 'fit'))
print('select_k has fit_transform: %s' % hasattr(select_k, 'fit_transform'))


#help(tfidf_vectorizer)

#result = sk.cross_validation.cross_val_score(pipeline, data[[0]], data[[1]])

#print(result)

True
False
False
select_k has transform: True
select_k has fit: True
select_k has fit_transform: True


In [26]:
pipe_factory = csspipe.PipeFactory()


In [27]:
tfidf_nb_pipe = pipe_factory.create_tfidf_nb()

In [28]:
tfidf_nb_pipe.steps[0]

('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words=None, strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
         vocabulary=None))

In [30]:
result = sklearn.cross_validation.cross_val_score(tfidf_nb_pipe, data[[0]].values, data[[1]].values)
print(result)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'